In [146]:
import pandas as pd
import numpy as np
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    IntegerType, StringType, IntegerType, FloatType, 
    StructField, StructType, DoubleType
)
from pyspark.ml.recommendation import ALS
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from pyspark.sql.functions import udf, col, isnan
import psycopg2
from pandas.io import sql
from sklearn.model_selection import train_test_split
from pyspark.ml.evaluation import RegressionEvaluator
from itertools import product

In [147]:
cd ../mixmaker/

/Users/brettashley/galvanize/dsi/00-Capstone/ml-mix-lab/mixmaker


In [148]:
import database_interaction
import model

In [149]:
df = db.get_table('connections_distinct')
data = sr.spark.createDataFrame(df)

In [150]:
train, test = data.randomSplit([0.8, 0.2])

In [221]:
sr = model.SongRecommender(itemCol='song_id',
                     userCol='sampled_by_song_id',
                     ratingCol='is_connected',
                     nonnegative=True,
                     regParam=0.01,
                     rank=200,
                     alpha=1)
db = database_interaction.DatabaseInteraction(db_name='mixmaker')

In [222]:
recommender = sr.fit(train)

In [223]:
preds = sr.transform(recommender, test)

In [224]:
sr.rmse(preds, True)

0.3129171378049292

In [159]:
preds_train = sr.transform(recommender, train)

In [154]:
sr.rmse(preds_train)

0.010383430160107161

In [50]:
preds.filter("prediction + 1 > prediction").count()

8301

In [51]:
preds.count()

17924

In [53]:
top_10_recs = recommender.recommendForAllUsers(10).toPandas()

In [57]:
len(top_10_recs)

32595

In [55]:
train.count()

72706

In [58]:
top_10_recs.head()

,sampled_by_song_id,recommendations
0,148,"[(12442, 1.12744140625), (8652, 1.113459467887..."
1,463,"[(35330, 1.220956802368164), (10775, 1.1660300..."
2,471,"[(35330, 1.220956802368164), (10775, 1.1660300..."
3,833,"[(10826, 1.5349119901657104), (21, 1.452880978..."
4,1088,"[(21, 1.823341965675354), (7742, 1.79762363433..."


In [59]:
top_10_recs.loc[0,'recommendations']

[Row(song_id=12442, rating=1.12744140625),
 Row(song_id=8652, rating=1.1134594678878784),
 Row(song_id=11, rating=1.1061460971832275),
 Row(song_id=8656, rating=1.1018706560134888),
 Row(song_id=12460, rating=1.100779414176941),
 Row(song_id=12435, rating=1.0952532291412354),
 Row(song_id=12434, rating=1.0906071662902832),
 Row(song_id=16502, rating=1.086485743522644),
 Row(song_id=8611, rating=1.08619225025177),
 Row(song_id=18069, rating=1.0819225311279297)]

In [80]:
for i, row in top_10_recs.head().iterrows():
    print(list([row[0]] * 10))
    print((pd.DataFrame(row['recommendations'])))

[148, 148, 148, 148, 148, 148, 148, 148, 148, 148]
       0         1
0  12442  1.127441
1   8652  1.113459
2     11  1.106146
3   8656  1.101871
4  12460  1.100779
5  12435  1.095253
6  12434  1.090607
7  16502  1.086486
8   8611  1.086192
9  18069  1.081923
[463, 463, 463, 463, 463, 463, 463, 463, 463, 463]
       0         1
0  35330  1.220957
1  10775  1.166030
2   6316  1.164571
3  29277  1.153856
4   5334  1.147096
5  10785  1.136678
6  35317  1.130334
7   5028  1.127996
8   8620  1.108108
9  18137  1.106425
[471, 471, 471, 471, 471, 471, 471, 471, 471, 471]
       0         1
0  35330  1.220957
1  10775  1.166030
2   6316  1.164571
3  29277  1.153856
4   5334  1.147096
5  10785  1.136678
6  35317  1.130334
7   5028  1.127996
8   8620  1.108108
9  18137  1.106425
[833, 833, 833, 833, 833, 833, 833, 833, 833, 833]
       0         1
0  10826  1.534912
1     21  1.452881
2   8609  1.437298
3   8626  1.422530
4  12460  1.418432
5  31326  1.408498
6   7742  1.408180
7  10775  1.39335

In [66]:
output = pd.DataFrame(columns=['user_song_id', 'item_song_id', 'rating'])

In [67]:
output

,user_song_id,item_song_id,rating


In [87]:
for i, row in top_10_recs.head().iterrows():
    df = pd.DataFrame(row[1], columns=['item_song_id', 'rating'])
    df['user_song_id'] = list([row[0]] * 10)
    output = pd.concat([output, df])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [94]:
recommender = sr.fit(data)

In [225]:
preds = sr.get_predictions_for_all_users(recommender)

In [226]:
db = database_interaction.DatabaseInteraction()

In [227]:
preds.columns

Index(['user_song_id', 'item_song_id', 'rating'], dtype='object')

In [112]:
','.join((preds.loc[:,'user_song_id'].values))

'[,1,4,8, ,1,4,8, ,1,4,8, ,.,.,., ,4,7,9,0,1, ,4,7,9,0,1, ,4,7,9,0,1,]'

In [263]:
query = """
        INSERT INTO predictions_temp (user_song_id, item_song_id, rating)
        SELECT %s, %s, %s
        """

In [228]:
for i, row in preds.head().iterrows():
    if i % 2 == 0:
        print(i)
    print(i, row[0], row[1], row[2])

0
0 148 21436 1.038189172744751
1 148 12048 1.0321450233459473
2
2 148 18369 1.030478835105896
3 148 7304 1.0290838479995728
4
4 148 6671 1.028789758682251


In [257]:
test = preds[:100].reset_index(drop=True)

In [259]:
preds_reset = preds.reset_index(drop=True)

In [261]:
len(preds_reset)

387800

In [262]:
for row in test.itertuples():
    if row[0] % 10 == 0:
        print(row[0])
    print(row[0], row[1], row[2], row[3])

0
0 148 21436 1.038189172744751
1 148 12048 1.0321450233459473
2 148 18369 1.030478835105896
3 148 7304 1.0290838479995728
4 148 6671 1.028789758682251
5 148 29570 1.0286991596221924
6 148 13272 1.0283613204956055
7 148 8358 1.0254446268081665
8 148 19837 1.0243165493011475
9 148 22236 1.0233592987060547
10
10 463 1 1.0012716054916382
11 463 517 0.9890486001968384
12 463 405 0.9888384342193604
13 463 463 0.9887073636054993
14 463 512 0.9886931777000427
15 463 274 0.9886022806167603
16 463 480 0.9885218739509583
17 463 329 0.9884659647941589
18 463 556 0.9884563088417053
19 463 337 0.9883512258529663
20
20 471 1 1.0010104179382324
21 471 471 0.9891318678855896
22 471 369 0.9835389852523804
23 471 329 0.9829203486442566
24 471 431 0.9826690554618835
25 471 512 0.9803186655044556
26 471 480 0.980256974697113
27 471 556 0.9802442193031311
28 471 405 0.9801980257034302
29 471 337 0.9800354838371277
30
30 496 1 1.001094937324524
31 496 496 0.9889931082725525
32 496 405 0.9848722815513611
33 

In [264]:
for row in preds_reset.head().itertuples():
    if row[0] % 5000 == 0:
        print(row[0])
    db.cur.execute(query, 
                   (row[1], row[2], row[3]))
    db.conn.commit()

0


In [266]:
for row in preds_reset.itertuples():
    if row[0] % 5000 == 0:
        print(row[0])
    db.cur.execute(query, 
                   (row[1], row[2], row[3]))
    db.conn.commit()

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000


0
2
4
6
8
